In [ ]:
#Importing libraries
import scipy
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import random


In [ ]:
#loading dataset

my_data_dir = 'C:/Users/shrut/OneDrive/Desktop/Eco Tech/archive/Garbage classification/Garbage classification' #enter path to dataset here 
os.listdir(my_data_dir) 



In [ ]:
test_path = my_data_dir+'/test/'
train_path = my_data_dir+'/train/'


In [ ]:
#sample image
categories = os.listdir(train_path)
random_category= random.choice(categories)
random_category_dir = os.path.join(train_path, random_category)
category_files = os.listdir(random_category_dir)
random_sample_filename = random.choice(category_files)
category_files = os.listdir(random_category_dir)

sample_path= os.path.join(random_category_dir, random_sample_filename)
sample_img=imread(sample_path)
plt.imshow(sample_img)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.10, 
                               height_shift_range=0.10, 
                               rescale=1/255, 
                               shear_range=0.1, 
                               zoom_range=0.1, 
                               horizontal_flip=True, 
                               fill_mode='nearest' 
                              )

In [ ]:
image_gen.flow_from_directory(test_path)

In [ ]:
image_gen.flow_from_directory(train_path)

In [ ]:
#loading libraries

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
image_shape=(32,32,1)

#model

model= Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its multi-class classification so we use sigmoid
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(patience=5, monitor='val_loss')

In [ ]:
batch_size = 16
image_shape=(32,32,1)
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical')

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

In [ ]:
train_image_gen.class_indices

In [ ]:

import warnings
import scipy
warnings.filterwarnings('ignore')
results = model.fit(train_image_gen,epochs=80,
                              validation_data=test_image_gen,
                             callbacks=[early_stop])

In [ ]:
from tensorflow.keras.preprocessing import image
y_pred = np.argmax(model.predict(test_image_gen), axis=-1)
# y_pred = np.round(y_pred).astype(int)
# pred_probabilities = model.predict_classes(test_image_gen)
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(test_image_gen.classes,y_pred))

In [ ]:
import cv2

# Read the sample image as grayscale
sample_img = cv2.imread(sample_path, cv2.IMREAD_GRAYSCALE)

# Resize the image to the desired input shape (32x32)
sample_img_resized = cv2.resize(sample_img, (32, 32))

# Reshape the image to (1, 32, 32, 1)
sample_img_reshaped = sample_img_resized.reshape(1, 32, 32, 1)

# Now you can make predictions with the reshaped grayscale image
pred = model.predict(sample_img_reshaped)

pred = np.argmax(pred)

if (pred==1):
  print ('trash')
elif(pred==0) :
  print ('plastic')


In [ ]:
import tensorflow

tensorflow.keras.models.save_model(model,'C:/Users/shrut/OneDrive/Desktop/Eco Tech')

converter= tensorflow.lite.TFLiteConverter.from_keras_model(model=model)
                                    
model_tflite=converter.convert()

open("trash_rec.tflite",'wb').write(model_tflite)